In [1]:
import pandas as pd 
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import xgboost


In [4]:
stroke_df = pd.read_csv('./healthcare-dataset-stroke-data.csv')
pd.set_option("display.max_rows", None)
stroke_df.head()
#stroke_df = stroke_df.astype({'age': 'int64'})
#stroke_df = stroke_df.astype({'avg_glucose_level': 'int64'})
stroke_df.isna().sum()
stroke_df['bmi'] = stroke_df['bmi'].fillna(0)
#stroke_df = stroke_df.astype({'bmi': 'int64'})
stroke_df.dtypes

id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [5]:
stroke_df.isna().sum()


id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [6]:
stroke_df.nunique()


id                   5110
gender                  3
age                   104
hypertension            2
heart_disease           2
ever_married            2
work_type               5
Residence_type          2
avg_glucose_level    3979
bmi                   419
smoking_status          4
stroke                  2
dtype: int64

In [7]:
stroke_df.drop(columns='id', inplace=True)
stroke_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,0.0,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [20]:
#replace bmi where value_counts is under 10 with other
bmi_counts = stroke_df['bmi'].value_counts()
bmi_lt_10 = bmi_counts.loc[bmi_counts < 10].index.tolist()
bmi_0 = bmi_counts.loc[bmi_counts == 201].index.tolist()
replace_bmi = bmi_lt_10 + bmi_0
replace_bmi
for x in replace_bmi:
    stroke_df['bmi'] = stroke_df['bmi'].replace(x,'Other')

stroke_df['bmi'].value_counts()

Other    963
28.7      41
28.4      38
27.6      37
27.7      37
26.1      37
26.7      37
27.3      36
23.4      36
27.0      35
26.9      34
26.4      34
25.1      34
25.5      33
23.5      31
24.8      31
28.9      31
30.3      30
29.4      30
28.3      30
26.5      30
31.4      30
22.2      30
26.6      29
28.1      29
24.2      29
27.5      29
29.1      29
25.3      28
27.1      28
24.1      28
28.0      28
32.3      28
24.0      28
27.9      28
31.5      27
24.9      27
28.6      27
21.5      27
29.7      27
30.9      27
23.0      27
26.2      27
25.0      27
28.5      27
30.0      27
24.3      26
28.8      26
29.5      26
30.1      26
24.5      26
29.6      26
29.2      26
29.0      26
31.1      26
29.9      26
25.4      26
22.7      25
26.0      25
22.8      25
28.2      25
32.8      25
33.1      25
20.1      25
27.2      24
23.9      24
23.6      24
30.5      24
32.1      24
35.8      24
31.8      24
25.9      24
25.8      24
30.7      23
20.4      23
26.3      23
29.8      23

In [21]:
#replace age where value_counts is under 40 with other
age_counts = stroke_df['age'].value_counts()
age_lt_40 = age_counts.loc[age_counts < 40].index.tolist()
for x in age_lt_40:
    stroke_df['age'] = stroke_df['age'].replace(x,'Other')

stroke_df['age'].value_counts()

Other    319
78.0     102
57.0      95
52.0      90
54.0      87
51.0      86
79.0      85
45.0      85
53.0      85
55.0      83
50.0      83
59.0      80
49.0      79
31.0      79
56.0      77
37.0      76
61.0      76
47.0      75
44.0      75
40.0      74
62.0      74
63.0      74
41.0      73
38.0      72
60.0      72
42.0      71
39.0      71
32.0      71
80.0      70
43.0      70
58.0      68
34.0      68
48.0      66
5.0       65
23.0      64
26.0      62
65.0      62
46.0      62
71.0      61
66.0      60
81.0      60
17.0      60
18.0      60
20.0      59
8.0       58
33.0      57
25.0      57
13.0      57
82.0      56
27.0      55
24.0      55
30.0      55
2.0       55
14.0      54
28.0      54
69.0      54
35.0      54
75.0      53
64.0      53
36.0      52
16.0      52
29.0      51
19.0      50
76.0      50
67.0      49
21.0      47
68.0      47
3.0       46
73.0      46
22.0      45
70.0      45
12.0      45
72.0      45
15.0      45
77.0      42
74.0      40
Name: age, d

In [9]:
#replace glucose where value_counts is under 2 with other 

In [34]:
df_dummies = pd.get_dummies(stroke_df)
X = df_dummies.drop('stroke', axis=1)
y = df_dummies['stroke']
avg_dummies = pd.get_dummies(stroke_df['avg_glucose_level'])
avg_dummies
df_dummies = df_dummies.drop(columns= 'avg_glucose_level')
merged_dummies = pd.concat([df_dummies, avg_dummies], axis=1)
merged_dummies.head()
X = merged_dummies
y = y
X.columns = X.columns.astype(str)


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [41]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [39]:
len(X_train_scaled[0])

4278

In [42]:
nn = tf.keras.models.Sequential()
number_of_input_features = len(X_train_scaled[0])
hidden_neuron1 = 100 
hidden_neuron2 = 50 
hidden_neuron3 = 30 

nn.add(tf.keras.layers.Dense(units = hidden_neuron1, activation='relu', input_dim = number_of_input_features))
nn.add(tf.keras.layers.Dense(units = hidden_neuron2, activation='sigmoid'))
nn.add(tf.keras.layers.Dense(units = hidden_neuron3, activation='sigmoid'))
nn.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               427900    
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 30)                1530      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 434,511
Trainable params: 434,511
Non-trainable params: 0
_________________________________________________________________


In [43]:
nn.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [44]:
nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
120/120 [==============================] - 3s 13ms/step - loss: 0.3313 - accuracy: 0.8620
Epoch 2/100
120/120 [==============================] - 2s 13ms/step - loss: 0.1584 - accuracy: 0.9528
Epoch 3/100
120/120 [==============================] - 2s 14ms/step - loss: 0.0746 - accuracy: 0.9765
Epoch 4/100
120/120 [==============================] - 2s 13ms/step - loss: 0.0303 - accuracy: 0.9997
Epoch 5/100
120/120 [==============================] - 2s 13ms/step - loss: 0.0182 - accuracy: 1.0000
Epoch 6/100
120/120 [==============================] - 1s 11ms/step - loss: 0.0126 - accuracy: 1.0000
Epoch 7/100
120/120 [==============================] - 2s 13ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 8/100
120/120 [==============================] - 1s 12ms/step - loss: 0.0073 - accuracy: 1.0000
Epoch 9/100
120/120 [==============================] - 1s 11ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 10/100
120/120 [==============================] - 1s 12ms/step - loss: 0.004

In [45]:
xgb_model = xgboost.XGBClassifier()
xgb_model = xgb_model.fit(X_train_scaled, y_train)
predictions = xgb_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predictions)
accuracy

1.0

In [46]:
forest_model = RandomForestClassifier(n_estimators= 398, random_state= 78)
forest_model = forest_model.fit(X_train_scaled, y_train)
predictions = forest_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predictions)
accuracy 

0.9984350547730829